In [1]:
import plotly.express as px
from jupyter_dash import JupyterDash
from datetime import datetime as dt
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import pandas as pd
import zipfile

In [2]:
# load data 
with zipfile.ZipFile('geo_out_july_2020_central_time.csv.zip', 'r') as zip_ref:
    zip_ref.extractall()
    
df = pd.read_csv('geo_out_july_2020_central_time.csv', index_col=0)
df['emdCardNumber'] = df['emdCardNumber'].str.upper()

In [3]:
# preview dataset
df.head()

,_id,incidentNumber,latitude,longitude,emdCardNumber,alarmDateTime,arrivalDateTime,weather,location.type,location.coordinates,respondingVehicles,fireZone,alarm_datetime,alarm_date,alarm_time,alarm_year
0,5c2e925395fad062a201f078,FFD18123100136570,36.168805,-86.786157,FFIRST,2019-01-01T03:47:20.840Z,2018-12-31T21:47:33.813-06:00,partly-cloudy-night,Point,"[-86.78615697,36.16880519]","[{""dispatchDateTime"":""2018-12-31T21:47:20.84-0...",2E,2018-12-31 21:47:20.840,2018-12-31,21:47:20.840000,2018
1,5c2e925395fad062a201f079,FFD18123100136575,36.076574,-86.711212,25B2B,2019-01-01T03:59:06.560Z,2018-12-31T22:07:14.927-06:00,partly-cloudy-night,Point,"[-86.7112123,36.0765745]","[{""dispatchDateTime"":""2018-12-31T21:59:06.56-0...",6B,2018-12-31 21:59:06.560,2018-12-31,21:59:06.560000,2018
2,5c2e925395fad062a201f07a,FFD18123100136578,36.305932,-86.678156,30B1,2019-01-01T04:13:40.527Z,2018-12-31T22:23:35.763-06:00,partly-cloudy-night,Point,"[-86.6781558,36.30593223]","[{""dispatchDateTime"":""2018-12-31T22:13:40.527-...",41F,2018-12-31 22:13:40.527,2018-12-31,22:13:40.527000,2018
3,5c2e925395fad062a201f07b,FFD18123100136579,36.178359,-86.750031,21B1T,2019-01-01T04:13:59.627Z,2018-12-31T22:18:40.017-06:00,partly-cloudy-night,Point,"[-86.75003128,36.17835937]","[{""dispatchDateTime"":""2018-12-31T22:13:59.627-...",14C,2018-12-31 22:13:59.627,2018-12-31,22:13:59.627000,2018
4,5c2e925495fad062a201f07c,FFD18123100136582,36.177560,-86.749953,25B6,2019-01-01T04:31:49.277Z,2018-12-31T22:36:04.02-06:00,partly-cloudy-night,Point,"[-86.74995287,36.17756044]","[{""dispatchDateTime"":""2018-12-31T22:31:49.277-...",14C,2018-12-31 22:31:49.277,2018-12-31,22:31:49.277000,2018


In [4]:
# build app
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H1("Incident Reports"),
    html.Label([
       "Date Range",
        dcc.DatePickerRange(
            id='date-picker-range',
            min_date_allowed=dt(2019, 1, 1),
            max_date_allowed=dt(2020, 7, 9),
            start_date=dt(2019, 1, 1).date(),
            end_date=dt(2020, 6, 30).date()
        ),
    ]),
    html.Div(),
    html.Label([
        "Radius",
        dcc.Slider(
            id='heatmap-radius',
            min=1,
            max=10,
            step=0.05,
            marks={i: '{}'.format(i) for i in range(1, 11)},
            value=3
        ) 
    ]), 
    html.Div(),
    html.Label([
        "Incident Categories",
           dcc.Checklist(
    options=[
          {'label': 'All', 'value': '1002'},
      {'label': 'Automatic Crash Notifications', 'value': '34'},
            {'label': 'Motor Vehicle Accidents', 'value': '29'},
        {'label': 'Breathing Problems', 'value': '6'},
        {'label': 'Burns', 'value': '7'},
        {'label': 'Cardiac Problems', 'value': '9'},
        {'label': 'Chest Pain', 'value': '10'},
             {'label': 'Stab/Gunshot', 'value': '27'},    
        {'label': 'Pandemic Flu', 'value': '36'},
        {'label': 'Structure Fire', 'value': '69'},
        {'label': 'Outside Fire', 'value': '67'},
        {'label': 'Vehicle Fire', 'value': '71'},      
    ],
    value=['29', '34'],
    id='emd-card-num-dropdown' 
   
)  
        
    ]),
    
     html.P(
                            """Select specific months to show data from."""
                        ),
    
    
           html.Div(
                                    className="div-for-dropdown",
                                    children=[
                                        # Dropdown to select times
                                        dcc.Dropdown(
                                            id="bar-selector",
                                            options=[
                                                {
                                                    "label": str(n),
                                                    "value": str(n),
                                                }
                                                for n in range(1,13)
                                            ],
                                            multi=True,
                                            placeholder="Select certain months",
                                        )
                                    ],
                                ),
          html.P(
                            """Heatmap of the incidents."""
                        ),
    
    dcc.Graph(
        id='heatmap',
        style={
            'height': 700
        }
    ),
      html.P(
                            """Select different the month from the historgram to narrow down the heatmap shown above."""
                        ),
    dcc.Graph(
        id='bar-chart',
        style={
            'height': 300
        }
    )
])



In [5]:
# Selected Data in the Histogram updates the Values in the DatePicker
@app.callback(
    Output("bar-selector", "value"),
    [Input("bar-chart", "selectedData"), Input("bar-chart", "clickData")],
)
def update_bar_selector(value, clickData):
    holder = []
    if clickData:
        holder.append(str(int(clickData["points"][0]["x"])))
    if value:
        for x in value["points"]:
            holder.append(str(int(x["x"])))
    return list(set(holder))

In [6]:
@app.callback(
    Output('heatmap', 'figure'),
    [Input("date-picker-range", "start_date"),
    Input("date-picker-range", "end_date"),
    Input("heatmap-radius", "value"),
    Input('emd-card-num-dropdown', 'value'),
    Input("bar-selector", "value")]
)
def update_heatmap(start_date, end_date, radius, emd_card_num, datemonth):
    if '1002' in emd_card_num:
        emd_card_num=range(1,136)        
    date_condition = ((df['alarm_date'] >= start_date) & (df['alarm_date'] <= end_date))
    string = '[A-Z]'
    updatedlist = [str(x) + string for x in emd_card_num]
    separator = '|'
    search_str = '^' + separator.join(updatedlist)
    emd_card_condition = (df.emdCardNumber.str.contains(search_str))
    result = df.loc[date_condition & emd_card_condition][['alarm_datetime','latitude','longitude']]  
    
    if len(datemonth)!=0:            
            result['month'] = pd.to_datetime(result['alarm_datetime']).dt.month
            month_condition = ((result['month'].isin(datemonth)))
            result = result.loc[month_condition][['latitude','longitude']]  
            
             
    return px.density_mapbox(result, lat='latitude', lon='longitude', radius=radius,
                        center=dict(lat=36.16228, lon=-86.774372), zoom=10,
                        mapbox_style="stamen-terrain")

In [7]:
@app.callback(
    Output('bar-chart', 'figure'),
    [Input("date-picker-range", "start_date"),
    Input("date-picker-range", "end_date"),
    Input('emd-card-num-dropdown', 'value')]
)
def update_bar_chart(start_date, end_date, emd_card_num):
    if '1002' in emd_card_num:
        emd_card_num=range(1,136)
    date_condition = ((df['alarm_date'] >= start_date) & (df['alarm_date'] <= end_date))
    string = '[A-Z]'
    updatedlist = [str(x) + string for x in emd_card_num]
    separator = '|'
    search_str = '^' + separator.join(updatedlist)
    emd_card_condition = (df.emdCardNumber.str.contains(search_str))
    
    result = df.loc[date_condition & emd_card_condition][['alarm_datetime']]    
    result['alarm_datetime'] = pd.to_datetime(result['alarm_datetime'])

    result['month_year'] = result['alarm_datetime'].dt.month 
    result['month_year'] = result['month_year'].astype(str)

    result = result.groupby(['month_year']).count().reset_index()
    result.columns = ['month', 'count']

    return px.bar(result, x='month', y='count', text='count')

In [8]:
# Run app and click link to display in separate tab
app.run_server(port=9100)

Dash app running on http://127.0.0.1:9100/
